In [1]:
#from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType
import haversine
from pyspark.sql import SparkSession
from pyspark.sql.functions import *


# Import PySpark Pandas
import pyspark.pandas as ps

import pandas as pd


In [2]:
spark_ = SparkSession.builder.appName('practice').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/09 15:56:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = spark_.read.option('header', True).csv('go_bike_with_distance.csv', inferSchema=True)
# uncal_data = spark_.read.option('header', True).csv('tripdata.csv', inferSchema=True)

In [4]:
#df = df.withColumnRenamed('distance', 'cal_distant')
df = df.drop('_c0') # removing unnecceray column

In [5]:
df.show(3) # view is not clear

+----------+--------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+-------+---------+-----------------+-------------+-----------+-----------------+
|start_time|end_time|start_station_id|  start_station_name|start_station_latitude|start_station_longitude|end_station_id|    end_station_name|end_station_latitude|end_station_longitude|bike_id|user_type|member_birth_year|member_gender|     pyment|Distance_in_meter|
+----------+--------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+-------+---------+-----------------+-------------+-----------+-----------------+
|   57:39.7| 12:50.2|              74|Laguna St at Haye...|           37.77643482|            -122.426244|            43|San Francisco Pub...|          37.7787677|         -122.4159292|     96| Customer

  #### tasks for this week:
1-calculate the total distance for each bike and list the top 10 <br>
2-calculate the number of trips for each start station list top 10 and find the ratio of using as male or female<br>
3-make a comparison to find the percentage of usage for customer and subscriber<br>
4-calculate the age of all users and show the relation between the distance and the age<br>
team leader : moihbur 
!! siraj will send the dataset with distance column !!

### 1-calculate the total distance for each bike and list the top 10

In [6]:
df.show(n=3,vertical=True) # make a clear view. RECORD repressnts row number

-RECORD 0---------------------------------------
 start_time              | 57:39.7              
 end_time                | 12:50.2              
 start_station_id        | 74                   
 start_station_name      | Laguna St at Haye... 
 start_station_latitude  | 37.77643482          
 start_station_longitude | -122.426244          
 end_station_id          | 43                   
 end_station_name        | San Francisco Pub... 
 end_station_latitude    | 37.7787677           
 end_station_longitude   | -122.4159292         
 bike_id                 | 96                   
 user_type               | Customer             
 member_birth_year       | 1987                 
 member_gender           | Male                 
 pyment                  | credit card          
 Distance_in_meter       | 942.93               
-RECORD 1---------------------------------------
 start_time              | 56:34.8              
 end_time                | 49:55.6              
 start_station_id   

In [7]:
df = df.withColumnRenamed('Distance_in_meter', 'dist_meter') # simply renaming column

In [8]:
df.select('bike_id').distinct().count() # checking how many unique values in bike_id -> 3673

3673

In [9]:
bike_distance = df.groupBy('bike_id').agg(sum('dist_meter').alias('total_meter')) # total distant for each bike
bike_distance.show()

+-------+------------------+
|bike_id|       total_meter|
+-------+------------------+
|   2366| 249465.2900000001|
|   2659|         490692.41|
|    496| 396983.3199999999|
|    471|         578013.06|
|   1238|         167757.94|
|   1591|         415585.59|
|    148|210349.24999999994|
|   2122|185749.78999999998|
|   2866|403550.14000000013|
|   1580|226680.16999999993|
|   1645|328620.45999999996|
|   1829|415100.30999999994|
|   1342|         167125.82|
|   2142|162044.23000000004|
|    833|15854.170000000002|
|    463|         108842.98|
|   1959|          219682.6|
|   1088|         523288.28|
|   3175|         258649.06|
|   2563|         196767.43|
+-------+------------------+
only showing top 20 rows



In [10]:
#bike_distance = df.groupBy('bike_id').agg({'dist_meter':'sum'}).alias('total_meter') # alias not working with dictionary

In [11]:
bike_distance.count() # same as before -> 3673

bike_distance.show()

+-------+------------------+
|bike_id|       total_meter|
+-------+------------------+
|   2366| 249465.2900000001|
|   2659|         490692.41|
|    496| 396983.3199999999|
|    471|         578013.06|
|   1238|         167757.94|
|   1591|         415585.59|
|    148|210349.24999999994|
|   2122|185749.78999999998|
|   2866|403550.14000000013|
|   1580|226680.16999999993|
|   1645|328620.45999999996|
|   1829|415100.30999999994|
|   1342|         167125.82|
|   2142|162044.23000000004|
|    833|15854.170000000002|
|    463|         108842.98|
|   1959|          219682.6|
|   1088|         523288.28|
|   3175|         258649.06|
|   2563|         196767.43|
+-------+------------------+
only showing top 20 rows



In [12]:
bike_distance.sort(desc('total_meter')).show(n=10)  # top 10 bikers with id using sort function

+-------+-----------------+
|bike_id|      total_meter|
+-------+-----------------+
|     68|        743369.39|
|   2178|        721182.22|
|    256|671915.9999999999|
|    235|670162.0900000001|
|   2049|        656828.08|
|    441|656642.3300000001|
|   2226|        647823.24|
|    796|        646867.81|
|    190|640294.2399999999|
|   2365|639412.8000000003|
+-------+-----------------+
only showing top 10 rows



### 2-calculate the number of trips for each start station list top 10 and find the ratio of using as male or female
#### calculate the number of trips for each start station list top 10
#### find the ratio of using as male or female

In [13]:
""""wild card charecter (*) referes every column in dataset. In this case, * apple the count on start_station_name column.
simple help(df.groupBy) will show how this function works"""

# calculate the number of trips for each start station to have a list of top 10

count_start_station = df.groupBy('start_station_name').agg(count('*').alias('num_station')) 

count_start_station.show()

+--------------------+-----------+
|  start_station_name|num_station|
+--------------------+-----------+
|19th St at Florid...|       1569|
|College Ave at Ha...|        733|
|5th St at Brannan St|       2055|
|Townsend St at 7t...|       6223|
|Valencia St at Ce...|       1665|
|Shattuck Ave at H...|        704|
|San Salvador St a...|        593|
|53rd St at Hollis St|        149|
|  12th St at 4th Ave|        993|
|Folsom St at 15th St|       1675|
|Spear St at Folso...|       4798|
|Washington St at ...|       2310|
|49th St at Telegr...|       1223|
|West Oakland BART...|       1162|
|El Embarcadero at...|       3386|
|Jackson St at 11t...|          2|
|San Francisco Fer...|      15187|
|The Embarcadero a...|       4269|
|59th St at Horton St|        268|
|Grand Ave at Perk...|       3000|
+--------------------+-----------+
only showing top 20 rows



In [14]:
# possible to change the column name
count_start_station.sort(desc('num_station')).show(n=10, truncate=False) # list top 10

+---------------------------------------------------------+-----------+
|start_station_name                                       |num_station|
+---------------------------------------------------------+-----------+
|San Francisco Ferry Building (Harry Bridges Plaza)       |15187      |
|The Embarcadero at Sansome St                            |13664      |
|San Francisco Caltrain (Townsend St at 4th St)           |12546      |
|San Francisco Caltrain Station 2  (Townsend St at 4th St)|12055      |
|Market St at 10th St                                     |11960      |
|Montgomery St BART Station (Market St at 2nd St)         |11334      |
|Berry St at 4th St                                       |10956      |
|Powell St BART Station (Market St at 4th St)             |10142      |
|Howard St at Beale St                                    |9926       |
|Steuart St at Market St                                  |9347       |
+---------------------------------------------------------+-----

In [15]:
ratio_gender = df.groupBy('start_station_name','member_gender' )\
.agg(count('*').alias('num_people'))  # separting the column 

ratio_gender.show()

+--------------------+-------------+----------+
|  start_station_name|member_gender|num_people|
+--------------------+-------------+----------+
|Lakeshore Ave at ...|        Other|        41|
| George St at 1st St|         Male|       100|
|Telegraph Ave at ...|        Other|        58|
|  Berry St at 4th St|        Other|        65|
|Shattuck Ave at 5...|        Other|         3|
|S. 4th St at San ...|        Other|         2|
|Shattuck Ave at 5...|         Male|       251|
| Julian St at 6th St|       Female|        25|
|Laguna St at Haye...|         Male|      1840|
|10th St at Fallon St|        Other|        62|
|14th St at Filber...|         Male|        18|
| Dover St at 57th St|        Other|        21|
|College Ave at Ha...|         null|        52|
|5th St at Brannan St|         null|       213|
|19th St at Florid...|         null|       112|
|Grand Ave at Perk...|         Male|      1736|
|      DeFremery Park|       Female|       169|
|Townsend St at 7t...|         null|    

In [31]:
ratio_gender.where((df.member_gender == 'Male') | (df.member_gender =='Female'))\
.sort('start_station_name').show(truncate=False) # extarcting rows with male and female rows

new_df = ratio_gender.where((df.member_gender == 'Male') | (df.member_gender =='Female'))\
.sort('start_station_name').toPandas()



+-----------------------+-------------+----------+
|start_station_name     |member_gender|num_people|
+-----------------------+-------------+----------+
|10th Ave at E 15th St  |Male         |104       |
|10th Ave at E 15th St  |Female       |43        |
|10th St at Fallon St   |Female       |424       |
|10th St at Fallon St   |Male         |871       |
|11th St at Bryant St   |Female       |164       |
|11th St at Bryant St   |Male         |1370      |
|11th St at Natoma St   |Male         |1487      |
|11th St at Natoma St   |Female       |342       |
|12th St at 4th Ave     |Male         |573       |
|12th St at 4th Ave     |Female       |155       |
|12th St at 6th Ave     |Female       |38        |
|12th St at 6th Ave     |Male         |162       |
|13th St at Franklin St |Female       |9         |
|13th St at Franklin St |Male         |44        |
|14th St at Filbert St  |Male         |18        |
|14th St at Filbert St  |Female       |44        |
|14th St at Mandela Pkwy|Female

In [45]:
new_df.groupby('member_gender').num_people.mean()

member_gender
Female     366.620818
Male      1280.580882
Name: num_people, dtype: float64

In [86]:
#m_fe = df.where((df.member_gender == 'Male')|(df.member_gender =='Female'))

# m_fe.groupBy('member_gender').count().show() #checking how many male and female are there 

ma_fe = df.where((df.member_gender == 'Male')|(df.member_gender =='Female')).groupBy('member_gender').count()

In [119]:
ma_fe = ma_fe.withColumnRenamed('count', 'mf_count')

total_ma_fa = ma_fe.agg(sum('mf_count')).alias('total_')
#ma_fe.withColumn('percent', col('count')

ma_fe.withColumn('percent', col('mf_count') / 446939).show()

total_ma_fa.show()

+-------------+--------+------------------+
|member_gender|mf_count|           percent|
+-------------+--------+------------------+
|       Female|   98621|0.2206587476143277|
|         Male|  348318|0.7793412523856723|
+-------------+--------+------------------+

+-------------+
|sum(mf_count)|
+-------------+
|       446939|
+-------------+



In [95]:
ma_fe.show()

+-------------+------+
|member_gender| count|
+-------------+------+
|       Female| 98621|
|         Male|348318|
+-------------+------+



### 3-make a comparison to find the percentage of usage for customer and subscriber

In [82]:
percent_user = df.groupBy('user_type').agg(count('*').alias('total'))

percent_user.withColumn('perc', col('total')/ df.count()).show()

+----------+------+-------------------+
| user_type| total|               perc|
+----------+------+-------------------+
|Subscriber|409230| 0.7874350586877045|
|  Customer|110470|0.21256494131229556|
+----------+------+-------------------+



## dont run 
ratio_gender.where(((df.start_station_name == 'San Francisco Ferry Building (Harry Bridges Plaza)') & \
                   (df.member_gender == 'Male') | (df.member_gender =='Female'))).show()

df.where((df.start_station_name == 'San Francisco Ferry Building (Harry Bridges Plaza)')\
         & ((df.member_gender == 'Male')|(df.member_gender =='Female'))).show(n=3,vertical=True) # cheking whether previous data is right !!

m_fe.groupBy('member_gender', 'start_station_name').count().show(truncate=False)

In [ ]:
df = df.withColumn('age_now', 2022 - col('member_birth_year')) # new frame with age 

df.show(n=2, vertical=True)

In [ ]:
df.select('dist_meter','age_now').where(df.dist_meter >1).show()

#df.where(df.dist_meter >1).show(vertical=True)

## just python

In [ ]:
import json
import requests as r

In [ ]:
k = r.get('https://www.google.com/finance/').json

json.load(k)